#### Introduction et objectifs du projet

##### <u>Contexte :</u>

L'analyse de sentiments est un domaine essentiel en science des données, en particulier en NLP. Elle utilise le machine learning et la linguistique pour détecter, extraire et étudier les émotions et les informations subjectives dans les textes sources.

Dans le contexte des avis de films, l'analyse de sentiments permet de comprendre les perceptions et les émotions des spectateurs à partir de leurs commentaires, ce qui est précieux pour les critiques de films, les studios de cinéma et les plateformes de streaming.

##### <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques. Les objectifs de l'application sont les suivants :

1. Permettre aux utilisateurs de saisir un titre de film ou son ID pour récupérer des avis pertinents.

2. Analyser les sentiments exprimés dans ces avis, en calculant la polarité (positivité ou négativité des sentiments) et la subjectivité (mesure de l'opinion personnelle par rapport à un fait objectif) des commentaires.

3. Comparer les "ratings" (notes) des films avec la polarité des avis pour voir si les évaluations numériques correspondent aux sentiments exprimés dans les commentaires.

4. Examiner les différences, le cas échéant, entre les sentiments exprimés avant et après la sortie du film, fournissant ainsi une perspective unique sur l'évolution des perceptions des spectateurs.

##### <u>Méthodes de récupération des données :</u>

Dans un premier temps, pour éviter de surcharger l'API de TMDB et de devoir gérer des bases de données trop volumineuses, nous avons décidé de limiter notre étude aux films sortis entre x et y. Une fois cette sélection effectuée, nous avons procédé à la récupération des données via 3 méthodes différentes :

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

L'objectif de cette section est de comparer les différentes méthodes de récupération des données afin de déterminer laquelle est la plus efficace et la plus pertinente pour notre projet.
Dans une première approche il était question de tester les 3 méthodes, voir les avantages et inconvénients de chacune d'entre elles, de savoir si elles étaient complémentaires ou non, et de déterminer laquelle était la plus efficace et la plus pertinente pour notre projet.

Dans un second temps l'objectif est de structurer les données récupérées afin de pouvoir les exploiter dans la suite du projet. Si possible par une seule des méthodes de récupération des données mentionnées ci-dessus.

Ensuite, il s'agira de mettre en place une méthode d'analyse de sentiments sur les commentaires récupérés, il existe des librairies python qui permettent de faire cela, mais il est également possible de créer notre propre méthode d'analyse de sentiments. 

Choix numéro 1 : Réaliser nos propres méthodes d'analyse de sentiments.

Choix numéro 2 : Utiliser des librairies python existantes tel que : TextBlob, NLTK.

Enfin il s'agira de mettre en place une application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.


#### <u>Partie 1 : import des librairies</u>

- package imdb à importer via le code ci-dessous : (important qu'il soit installé sur la machine et qu'il soit à jour)

In [ ]:
#pip install git+https://github.com/santhoshse7en/imdb

In [3]:
#from imdb import IMDb
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd

- Appel de la clé API de TMDB 

In [4]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

Première étape : on va tester notre procédé sur un seul film, pour voir si ça fonctionne, et si oui, on pourra ensuite l'appliquer à tous les films de notre base de données.

The Shawshank Redemption : ID sur le site de TMDB : tt0111161

In [7]:
film_id = 'tt0111161'

url_date_release = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&language=en-US"

response = requests.get(url_date_release)

if response.status_code == 200:
    film_details = response.json()
    date_sortie_str = film_details.get("release_date", "")
    if date_sortie_str:
        date_sortie = datetime.strptime(date_sortie_str, "%Y-%m-%d")
        print(f"Date de sortie du film : {date_sortie}")
    else:
        print("Date de sortie non trouvée")
else:
    print("Erreur lors de la requête :", response.status_code)

url_film = f"https://www.imdb.com/title/{film_id}/reviews" # Récupérer les avis sur IMDb

response = requests.get(url_film)

reviews_dict = {}

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    review_elements = soup.find_all('div', class_='review-container')
    for element in review_elements:
        review_text = element.find('div', class_='text show-more__control').get_text().strip()
        review_date_str = element.find('span', class_='review-date').get_text().strip()
        review_date = datetime.strptime(review_date_str, '%d %B %Y').date()
        reviews_dict[review_date] = review_text
else:
    print("Erreur lors de la requête :", response.status_code)

print("Nombre d'avis pour ce film :",len(reviews_dict)) 